# WEEK_03

## 1. 데이터 스케일링 변환

- Scaler 객체를 이용해 변환
    - fit() : 데이터 변환을 위한 기준 정보를 설정함 (ex : 최댓값 / 최솟값 설정 등)
    - transform() : 설정된 정보를 이용해 데이터를 변환
    - fit_transform() : fit 과 transform을 한번에 적용
- 변환 시 유의할 점
    - 학습 / 테스트 데이터 세트에 변환 시 학습 데이터에 fit() 한 동일한 스케일러로 테스트 데이터에 적용해야 한다.
        
        → 만약 테스트 데이터에 새로 fit()한 스케일러로 예측을 수행하게 되면 학습 데이터와 테스트 데이터의        스케일링 기준 정보가 서로 달라지기 때문에 올바른 예측결과를 도출하지 못할 수 있음.
        
- Scaler class의 fit(), transform()은 2차원 이상 데이터만 가능함.

In [2]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# data 2차원 reshape
train_array = np.arange(0, 11).reshape(-1, 1)
test_array = np.arange(0, 6).reshape(-1, 1)

# MinMaxScaler 객체에 별도의 feature_range 파라미터 값을 지정하지 않으면 0~1 값으로 변환 
scaler = MinMaxScaler()

# fit -> min=0, max=10
scaler.fit(train_array)

# transform -> 원본 10->1로 변경됨, 1/10 scale로 테스트 데이터 변환
train_scaled = scaler.transform(train_array)

print('original train data:', np.round(train_array.reshape(-1)),2)


original train data: [ 0  1  2  3  4  5  6  7  8  9 10] 2
